In [9]:
from confluent_kafka import Producer, Consumer

In [22]:
consumer = KafkaConsumer('quickstart', auto_offset_reset='earliest')
consumer.

In [9]:
import mysql.connector

cnx = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="my-secret-pw"
)

print(cnx)

In [11]:
cursor = cnx.cursor()

In [1]:
from tortoise.models import Model
from tortoise import fields

In [1]:
"""
This example demonstrates most basic operations with single model
"""
from tortoise import Tortoise, fields, run_async
from tortoise.models import Model


class Event(Model):
    id = fields.IntField(pk=True)
    name = fields.TextField()
    datetime = fields.DatetimeField(null=True)

    class Meta:
        table = "event"

    def __str__(self):
        return self.name


async def run():
    await Tortoise.init(db_url="sqlite://:memory:", modules={"models": ["__main__"]})
    await Tortoise.generate_schemas()

    event = await Event.create(name="Test")
    await Event.filter(id=event.id).update(name="Updated name")

    print(await Event.filter(name="Updated name").first())
    # >>> Updated name

    await Event(name="Test 2").save()
    print(await Event.all().values_list("id", flat=True))
    # >>> [1, 2]
    print(await Event.all().values("id", "name"))
    # >>> [{'id': 1, 'name': 'Updated name'}, {'id': 2, 'name': 'Test 2'}]


# if __name__ == "__main__":
#     run_async(run())


In [3]:
await Tortoise.init(db_url="sqlite://:memory:", modules={"models": ["__main__"]})
await Tortoise.generate_schemas()

event = await Event.create(name="Test")
await Event.filter(id=event.id).update(name="Updated name")

print(await Event.filter(name="Updated name").first())

Updated name


In [3]:
from tortoise import Tortoise

async def init():
    # Here we connect to a SQLite DB file.
    # also specify the app name of "models"
    # which contain models from "app.models"
    await Tortoise.init(
        db_url='sqlite://db.sqlite3',
        modules={'models': ['app.models']}
    )
    # Generate the schema
    await Tortoise.generate_schemas()

In [7]:
tournament = Tournament(name='New Tournament')

In [8]:
tournament.save()

<coroutine object Model.save at 0x7fb8f2f77240>

In [5]:
# Create instance by save
tournament = Tournament(name='New Tournament')
await tournament.save()

# Or by .create()
await Event.create(name='Without participants', tournament=tournament)
event = await Event.create(name='Test', tournament=tournament)
participants = []
for i in range(2):
    team = await Team.create(name='Team {}'.format(i + 1))
    participants.append(team)

# M2M Relationship management is quite straightforward
# (also look for methods .remove(...) and .clear())
await event.participants.add(*participants)

# You can query a related entity with async for
async for team in event.participants:
    pass

# After making a related query you can iterate with regular for,
# which can be extremely convenient when using it with other packages,
# for example some kind of serializers with nested support
for team in event.participants:
    pass


# Or you can make a preemptive call to fetch related objects
selected_events = await Event.filter(
    participants=participants[0].id
).prefetch_related('participants', 'tournament')

# Tortoise supports variable depth of prefetching related entities
# This will fetch all events for Team and in those events tournaments will be prefetched
await Team.all().prefetch_related('events__tournament')

# You can filter and order by related models too
await Tournament.filter(
    events__name__in=['Test', 'Prod']
).order_by('-events__participants__name').distinct()

TypeError: 'NoneType' object is not iterable

In [21]:
from tortoise import Tortoise, run_async

async def init():
    # Here we create a SQLite DB using file "db.sqlite3"
    #  also specify the app name of "models"
    #  which contain models from "app.models"
    await Tortoise.init(
        db_url='sqlite://db.sqlite3',
        modules={'models': ['app.models']}
    )
    # Generate the schema
    await Tortoise.generate_schemas()

# run_async is a helper function to run simple async Tortoise scripts.
run_async(init())

ImportError: cannot import name 'run_async' from 'tortoise' (/opt/miniconda3/envs/internet-of-trams/lib/python3.9/site-packages/tortoise/__init__.py)

In [19]:
query = """
CREATE TABLE IF NOT EXISTS
	sys.trams
	(tram_id INT(10)
    ,tram_name VARCHAR (20)
    ,line INT(5))
    """

cursor.execute(query)

In [18]:
query = """
DROP TABLE sys.trams
    """

cursor.execute(query)

In [3]:
from kafka import KafkaConsumer
consumer = KafkaConsumer('quickstart', auto_offset_reset='latest')
messages = []
for msg in consumer:
    print(msg)

ConsumerRecord(topic='quickstart', partition=0, offset=1894, timestamp=1710001039679, timestamp_type=0, key=None, value=b'infromacja0', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=11, serialized_header_size=-1)
ConsumerRecord(topic='quickstart', partition=0, offset=1895, timestamp=1710001039679, timestamp_type=0, key=None, value=b'infromacja1', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=11, serialized_header_size=-1)
ConsumerRecord(topic='quickstart', partition=0, offset=1896, timestamp=1710001039679, timestamp_type=0, key=None, value=b'infromacja2', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=11, serialized_header_size=-1)
ConsumerRecord(topic='quickstart', partition=0, offset=1897, timestamp=1710001039679, timestamp_type=0, key=None, value=b'infromacja3', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=11, serialized_header_size=-1)
ConsumerRecord(topic='quickstart', parti

KeyboardInterrupt: 